# **LSTM (Deep Learning Model)**

**STEP 1: Load Data (First Cell)**

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("solar_cleaned.csv")
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])
df = df.sort_values('DATE_TIME').reset_index(drop=True)

df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'solar_cleaned.csv'

**STEP 2: Select Target Only (LSTM Best Practice)**

In [ ]:
data = df[['AC_POWER']].values


**STEP 3: Scale Data (MANDATORY for LSTM)**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)


**STEP 4: Create Sequences (CORE STEP)**

In [ ]:
def create_sequences(data, seq_length=24):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

X, y = create_sequences(data_scaled, seq_length=24)

X.shape, y.shape


**STEP 5: Train–Test Split (Time-Series Correct)**

In [ ]:
split = int(len(X) * 0.8)

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


**STEP 6: Build LSTM Model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(X_train.shape[1], 1)),
    Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mse'
)

model.summary()


**STEP 7: Train LSTM**

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.1,
    verbose=1
)


**STEP 8: Predict & Inverse Scale**

In [ ]:
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)


**STEP 9: Evaluate LSTM**

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))

print("LSTM MAE:", mae)
print("LSTM RMSE:", rmse)


**STEP 10: Plot Results (Zoomed)**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,4))
plt.plot(y_test_inv[:500], label='Actual')
plt.plot(y_pred_inv[:500], label='Predicted')
plt.legend()
plt.title("LSTM – Actual vs Predicted AC Power")
plt.show()


**Save Files**

In [ ]:
import os
import pandas as pd

base_path = r"C:\Users\lalit\Desktop\solar_wind_ai_project"
output_path = os.path.join(base_path, "outputs")

print("Saving to:", output_path)

os.makedirs(output_path, exist_ok=True)

lstm_y_test_path = os.path.join(output_path, "y_test_lstm.csv")
lstm_pred_path   = os.path.join(output_path, "lstm_predictions.csv")

pd.DataFrame(y_test_inv).to_csv(lstm_y_test_path, index=False)
pd.DataFrame(y_pred_inv).to_csv(lstm_pred_path, index=False)

print("Saved files:")
print(os.listdir(output_path))


In [ ]:
**